In [1]:
# You can use python inline. E.g. you can define functions to be used later:

# import environment and expose in jupyter env
# this only works when shellvars is installed as it is in L-CAS Juptyer images
# https://github.com/LCAS/jupyterhub-deploy-docker/blob/master/singleuser/Dockerfile#L100

import shellvars
from os import environ

def loadenv(filename='/opt/ros/kinetic/setup.bash'):
    vs = shellvars.get_vars(filename)
    for v in vs:
        print('%s=%s' % (v.decode("utf-8"), str(vs[v].decode("utf-8"))))
        environ[v.decode("utf-8")] = str(vs[v].decode("utf-8"))

# you can call these functions easily
# (This example is a good way to load environment variables from a config file)

loadenv('/opt/ros/kinetic/setup.bash')

ROS_ROOT=/opt/ros/kinetic/share/ros
ROS_PACKAGE_PATH=/opt/ros/kinetic/share
ROS_MASTER_URI=http://localhost:11311
ROS_VERSION=1
LD_LIBRARY_PATH=/opt/ros/kinetic/lib:/opt/ros/kinetic/lib/x86_64-linux-gnu
PATH=/opt/ros/kinetic/bin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
ROS_DISTRO=kinetic
PYTHONPATH=/opt/ros/kinetic/lib/python2.7/dist-packages
PKG_CONFIG_PATH=/opt/ros/kinetic/lib/pkgconfig:/opt/ros/kinetic/lib/x86_64-linux-gnu/pkgconfig
CMAKE_PREFIX_PATH=/opt/ros/kinetic
ROS_ETC_DIR=/opt/ros/kinetic/etc/ros


In [2]:
# install marc-hanheide's own python3 version of rosbag_panda: https://github.com/marc-hanheide/RosbagPandas
#   - clone it
#   - run pip install --user -U . in it 
# tutorial at https://nimbus.unl.edu/2014/11/using-rosbag_pandas-to-analyze-rosbag-files/
#sudo apt-get install ros-kinetic-rosbag python-roslz4
def expand_sys_path():
    import sys
    from os import environ

    extra_paths = environ['PYTHONPATH'].split(':')
    sys.path.extend(extra_paths)

    new_path = []
    for p in sys.path:
        if not p in new_path:
            new_path.append(p)

    sys.path = new_path


expand_sys_path()

import rosbag
import pixiedust
from metakernel.display import display
from pprint import pprint


Failed to load Python extension for LZ4 support. LZ4 compression will not be available.


Pixiedust database opened successfully


In [116]:
import pandas as pd
import numpy as np

def bag2df(bagfile):

    bag = rosbag.Bag(bagfile)
    topics = bag.get_type_and_topic_info()[1]

    #for t in topics:
    #    print(t, topics[t])

    import rosbag_pandas
    dfs = {
       'qtc': rosbag_pandas.bag_to_dataframe(bagfile, include=['/robot5/online_qtc_creator/qtc_array']),
       'people_pose': rosbag_pandas.bag_to_dataframe(bagfile, include=['/robot5/human_perception/tracked_persons']),
       'robot_pose': rosbag_pandas.bag_to_dataframe(bagfile, include=['/robot5/robot_pose_odom']),
    }
    dfs['qtc']['ts'] = (
        dfs['qtc']['/robot5/online_qtc_creator/qtc_array/header/stamp/secs'] +
        (dfs['qtc']['/robot5/online_qtc_creator/qtc_array/header/stamp/nsecs'] / 10**9)
    )
    dfs['people_pose']['ts'] = (
        dfs['people_pose']['/robot5/human_perception/tracked_persons/header/stamp/secs'] +
        (dfs['people_pose']['/robot5/human_perception/tracked_persons/header/stamp/nsecs'] / 10**9)
    )
    dfs['robot_pose']['ts'] = (
        dfs['robot_pose']['/robot5/robot_pose_odom/header/stamp/secs'] +
        (dfs['robot_pose']['/robot5/robot_pose_odom/header/stamp/nsecs'] / 10**9)
    )

    #for d in dfs:
    #    df[d].set_index('ts', drop=False, inplace=True)
    #    df[d].sort_index(inplace=True)

    
#    df = rosbag_pandas.bag_to_dataframe(bagfile, include=['/robot5/online_qtc_creator/qtc_array'])
    return dfs


import glob
import re
from collections import defaultdict

df = defaultdict(dict)

rexpr = re.compile('bags/S([0-9])-T([0-9]\.[0-9])-A(\d*)_tiny_.*')
for filepath in glob.iglob('bags/*.bag'):
    s = rexpr.match(filepath)
    (subject, task, repetition) = s.groups()
    #print(filepath)
    #print(subject, task, repetition)
    try:
        df[task][repetition] = bag2df(filepath)
        print('read %s' % filepath)
    except Exception as e:
        print('*** error in processing %s: %s' % (filepath, str(e)))

#df = bag2df('bags/S1-T1.1-A1_tiny_2019-02-05-19-31-12.bag')


read bags/S1-T2.1-A1_tiny_2019-02-06-18-43-32.bag
read bags/S1-T1.1-A2_tiny_2019-02-06-18-36-33.bag
read bags/S1-T4.2-A1_tiny_2019-02-06-19-06-45.bag
read bags/S1-T2.1-A2_tiny_2019-02-06-18-45-51.bag
read bags/S1-T1.1-A1_tiny_2019-02-06-18-34-14.bag
read bags/S1-T2.2-A2_tiny_2019-02-06-18-50-29.bag
read bags/S1-T3.1-A2_tiny_2019-02-06-18-55-08.bag
read bags/S1-T1.2-A2_tiny_2019-02-06-18-41-12.bag
read bags/S1-T2.2-A1_tiny_2019-02-06-18-48-10.bag
read bags/S1-T1.2-A1_tiny_2019-02-06-18-38-52.bag
read bags/S1-T3.2-A1_tiny_2019-02-06-18-57-27.bag
read bags/S1-T4.2-A2_tiny_2019-02-06-19-09-04.bag
read bags/S1-T3.2-A2_tiny_2019-02-06-18-59-48.bag
read bags/S1-T3.1-A1_tiny_2019-02-06-18-52-49.bag
read bags/S1-T4.1-A2_tiny_2019-02-06-19-04-25.bag
read bags/S1-T4.1-A1_tiny_2019-02-06-19-02-06.bag


In [143]:
from yaml import load
#df['people_pose'].info()
#display(df['people_pose'])

for task in df:
    for rep in df[task]:
        try:
            c = df[task][rep]

            df_qtc_people = pd.merge_asof(
                c['qtc'], c['people_pose'])
            df_all = pd.merge_asof(
                df_qtc_people, c['robot_pose'])

            #display(df_all)
            #pprint(df_all.columns.tolist())

            start_time=min(df_all['ts'])
            df_all['time'] = df_all['ts'] - start_time

            for p in range(0,9):
                try: 
                    df_selected = df_all[
                        [
                            'time',
                            'ts',
                            '/robot5/online_qtc_creator/qtc_array/qtc/%d/qtc_robot_human' % p,
                            '/robot5/human_perception/tracked_persons/tracks/%d/pose/pose/position/x' % p,
                            '/robot5/human_perception/tracked_persons/tracks/%d/pose/pose/position/y' % p,
                            '/robot5/robot_pose_odom/pose/position/x',
                            '/robot5/robot_pose_odom/pose/position/y']
                    ]
                    df_selected.dropna(inplace=True)
                    qtc_seq = df_selected[-1:]['/robot5/online_qtc_creator/qtc_array/qtc/%d/qtc_robot_human' % p].tolist()
                    #print(type(qtc_seq))
                    print('task: %s (user: %d), rep: %s: %s' % (task, p, rep, qtc_seq))
                except:
                    pass
            #display(df_selected)
        except Exception as e:
            print('*** error in %s/%s: %s' % (task, rep, str(e)))





/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


task: 2.1 (user: 0), rep: 1: ['["-,0,+,+", "0,0,0,+", "0,-,0,+", "0,0,0,0", "0,+,0,-", "0,+,0,0", "0,+,0,+", "0,0,0,+"]']
task: 2.1 (user: 1), rep: 1: ['["-,-,+,+", "-,0,+,0", "-,+,+,-", "-,0,+,-", "-,-,+,-", "-,-,+,0", "-,-,+,+"]']
task: 2.1 (user: 2), rep: 1: ['["0,-,+,+", "0,-,+,0", "0,-,+,-", "0,-,+,0", "-,-,+,+", "-,0,+,0", "-,+,+,-", "-,0,+,-", "-,-,+,-", "-,-,+,0", "-,-,+,+"]']
task: 2.1 (user: 3), rep: 1: ['["0,-,+,+", "0,-,+,0", "0,-,+,-", "0,-,+,0", "-,-,+,+", "-,0,+,0", "-,+,+,-", "-,0,+,-", "-,-,+,-"]']
task: 2.1 (user: 0), rep: 2: ['["0,0,-,+", "0,0,0,+", "0,+,0,+", "0,+,0,0", "0,0,0,0"]']
task: 1.1 (user: 0), rep: 2: ['["-,-,+,+", "0,-,+,+", "-,-,+,+", "0,-,+,+", "-,-,+,+"]']
task: 1.1 (user: 1), rep: 2: ['["0,-,+,+", "-,-,+,+", "0,-,+,+"]']
task: 1.1 (user: 0), rep: 1: ['["+,-,+,+", "0,-,+,+", "-,-,+,+"]']
task: 4.2 (user: 0), rep: 1: ['["0,0,+,0", "+,0,+,0", "0,0,+,0", "-,0,+,0", "0,0,+,0", "+,0,+,0", "+,0,0,0", "+,0,-,0"]']
task: 4.2 (user: 0), rep: 2: ['["+,0,+,0"]']
